# PX action of equity + etfs

In [2]:
import requests

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=1QAPY627ICJSPO8R'

r = requests.get(url)

In [3]:
data = r.json()
print(data)

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-11-13 19:55:00', '4. Interval': '5min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2025-11-13 19:55:00': {'1. open': '304.8600', '2. high': '305.0000', '3. low': '304.8600', '4. close': '305.0000', '5. volume': '100'}, '2025-11-13 19:50:00': {'1. open': '305.2000', '2. high': '305.2000', '3. low': '304.8726', '4. close': '304.8726', '5. volume': '90'}, '2025-11-13 19:45:00': {'1. open': '305.1900', '2. high': '305.2000', '3. low': '305.0000', '4. close': '305.2000', '5. volume': '19'}, '2025-11-13 19:40:00': {'1. open': '305.1900', '2. high': '305.1900', '3. low': '305.0000', '4. close': '305.1900', '5. volume': '32'}, '2025-11-13 19:35:00': {'1. open': '305.1900', '2. high': '305.2000', '3. low': '304.6358', '4. close': '304.6358', '5. volume': '50'}, '2025-11-13 19:30:00': {'1. open': '305.0375', '2. high'

In [4]:
import pandas as pd
import datetime
ticker = data['Meta Data']['2. Symbol']
del data['Meta Data']
data = data['Time Series (5min)'] #needlesly nested


In [5]:
df = pd.DataFrame.from_dict(data).T
df = df.apply(pd.to_numeric).reset_index()
df = df.rename(columns={'1. open' : 'open',
           '2. high' : 'high',
           '3. low' : 'low',
           '4. close' :'close',
           '5. volume' : 'volume',
           'index' : 'datetime'})
df

,datetime,open,high,low,close,volume
0,2025-11-13 19:55:00,304.8600,305.0000,304.8600,305.0000,100
1,2025-11-13 19:50:00,305.2000,305.2000,304.8726,304.8726,90
2,2025-11-13 19:45:00,305.1900,305.2000,305.0000,305.2000,19
3,2025-11-13 19:40:00,305.1900,305.1900,305.0000,305.1900,32
4,2025-11-13 19:35:00,305.1900,305.2000,304.6358,304.6358,50
...,...,...,...,...,...,...
95,2025-11-13 11:55:00,308.9700,309.6800,307.9600,309.4650,81087
96,2025-11-13 11:50:00,309.4400,309.8552,308.8800,308.9700,22142
97,2025-11-13 11:45:00,309.2235,309.6800,309.1700,309.5730,60104
98,2025-11-13 11:40:00,309.8300,309.9400,309.0440,309.2350,27966


In [6]:
df['datetime'] = pd.to_datetime(df['datetime'])

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute

df['epoch'] = df['datetime'].astype('int64') // 10**9
df = (df
      .set_index('epoch')
      .reindex(columns=[ 'year', 'month', 'day', 'hour', 'minute', 
                'open', 'high', 'low', 'close']))


In [7]:
df

,year,month,day,hour,minute,open,high,low,close
epoch,,,,,,,,,
1763063700,2025,11,13,19,55,304.8600,305.0000,304.8600,305.0000
1763063400,2025,11,13,19,50,305.2000,305.2000,304.8726,304.8726
1763063100,2025,11,13,19,45,305.1900,305.2000,305.0000,305.2000
1763062800,2025,11,13,19,40,305.1900,305.1900,305.0000,305.1900
1763062500,2025,11,13,19,35,305.1900,305.2000,304.6358,304.6358
...,...,...,...,...,...,...,...,...,...
1763034900,2025,11,13,11,55,308.9700,309.6800,307.9600,309.4650
1763034600,2025,11,13,11,50,309.4400,309.8552,308.8800,308.9700
1763034300,2025,11,13,11,45,309.2235,309.6800,309.1700,309.5730


In [8]:
def getPxAction(ticker : str, api_key : str, interval : str = '5min') -> pd.DataFrame:
	"""
	Call the api to get a certain ticker as a df,
	with index as epoch timestamps, ohlc, and 
	year, month, day, hour, minute of the price
	----
	Usage:
	>>> getPxAction('IBM', 'demo').head(1)
	epoch       |year   |month  |day    |hour   |minute |open   |high   |low    |close  |
	=====================================================================================
	1759521300  |2025   |10     |03     |19     |55     |288.39 |288.75 |288.39 |288.45 |
	"""

	#the api call
	url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={api_key}&extended_hours=false'

	r = requests.get(url)
	data = r.json()[f'Time Series ({interval})']

	df = pd.DataFrame.from_dict(data).T
	df = df.apply(pd.to_numeric).reset_index()
	df = df.rename(columns={'1. open' : 'open',
			'2. high' : 'high',
			'3. low' : 'low',
			'4. close' :'close',
			'5. volume' : 'volume',
			'index' : 'datetime'})
	df['datetime'] = pd.to_datetime(df['datetime'])
	df['year'] = df['datetime'].dt.year
	df['month'] = df['datetime'].dt.month
	df['day'] = df['datetime'].dt.day
	df['hour'] = df['datetime'].dt.hour
	df['minute'] = df['datetime'].dt.minute
	df['epoch'] = df['datetime'].astype('int64') // 10**9
	df = (df
		.set_index('epoch')
		.reindex(columns=[ 'year', 'month', 'day', 'hour', 'minute', 
					'open', 'high', 'low', 'close']))

	return df

In [49]:
getPxAction('IBM', 'demo').head(2)

KeyError: 'Time Series (5min)'

In [ ]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo&datatype=csv'


In [56]:
pd.read_csv(url)

,timestamp,open,high,low,close,volume
0,2025-10-03 19:55:00,288.3900,288.75,288.3900,288.450,93
1,2025-10-03 19:50:00,288.3813,288.50,288.3800,288.400,56
2,2025-10-03 19:45:00,288.3805,288.74,288.3800,288.740,156
3,2025-10-03 19:40:00,288.3807,288.75,288.3800,288.750,141
4,2025-10-03 19:35:00,288.3800,288.38,288.3800,288.380,212
...,...,...,...,...,...,...
95,2025-10-03 11:55:00,291.1800,291.70,291.0700,291.615,31303
96,2025-10-03 11:50:00,291.2900,291.39,291.0900,291.155,26062
97,2025-10-03 11:45:00,290.8400,291.24,290.6900,291.240,22216
98,2025-10-03 11:40:00,291.0200,291.02,290.4700,290.910,43365


# ETF Constit

In [71]:
url = 'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol=QQQ&apikey=demo'

r = requests.get(url)

data = r.json()

In [62]:
div_yield = data['dividend_yield']
inc_date = data['inception_date']
data = data['holdings']


In [69]:
df = pd.json_normalize(data)
df = df.rename(columns={'symbol' : 'ticker',
           'description' : 'name'})
df['weight'] = df['weight'].astype('float')

In [73]:
data['net_expense_ratio']

'0.002'